In [2]:
from docplex.mp.model import Model
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import distance_matrix as dm
import numpy as np
import pandas as pd
import os

TARGET_WEEKDAY = 0  # 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
TARGET_HOUR = 8  # 0 = 00:00, 1 = 01:00, ..., 23 = 23:00

load_dotenv()

api_key = os.getenv("GOOGLE_MAPS_API_KEY")

matrix_time_first_tw, matrix_distance = dm.main(TARGET_WEEKDAY, TARGET_HOUR, api_key)
matrix_time_second_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 1, api_key
)
matrix_time_third_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 2, api_key
)
matrix_time_fourth_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 3, api_key
)

# import random
# # Dimensione della matrice
# n = 20
# # Inizializza una matrice vuota
# matrix_time_first_tw = [[0 if i == j else random.randint(5, 40) for j in range(n)] for i in range(n)]
# # Rendi la matrice simmetrica
# for i in range(n):
#     for j in range(i+1, n):
#         matrix_time_first_tw[j][i] = matrix_time_first_tw[i][j]
# # Stampa la matrice
# for row in matrix_time_first_tw:
#     print(row)



# sava datas in a matrix of matrixes
first_data = matrix_time_first_tw
second_data = matrix_time_first_tw
third_data = matrix_time_first_tw
fourth_data = matrix_time_first_tw
data = [first_data, second_data, third_data, fourth_data]

# parameters for time model
c = 20  # number of nodes
d = [8, 9, 10, 11, 12]  # time intervals
tw = [
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
]  # time windows
q = [
    0,
    10,
    10,
    -10,
    10,
    10,
    -10,
    10,
    10,
    -10,
    10,
    10,
    10,
    -10,
    10,
    10,
    10,
    10,
    -10,
    10,
]  # quantity of goods to collect/deliver
Q = 300  # maximum capacity of the vehicle
t_mv = 0.5  # time required to move one item in/out from a vehicle [min]
M = 100000  # big M
tot_q = 0  # total quantity of goods to collect/delivery
del_q = 0  # quantity of goods to deliver
v = 20  # maximum number of vehicles that can be used by the model

for good in q:
    tot_q += abs(good)

tot_t_mv = tot_q * t_mv  # total time required to move all goods in/out from the vehicle

# define model
mdl = Model("RT-VRP")

# decision variables
x = mdl.binary_var_dict(
    (
        (i, j, t, k)
        for i in range(c)
        for j in range(c)
        for t in range(len(d) - 1)
        for k in range(v)
    ),
    name="x",
)  # x == 1 if it is true that we are going from node i to node j at time t with vehicle k
T = mdl.continuous_var_matrix(
    c + 1, v, name="t"
)  # arrival time at node i for vehicle k
C = mdl.continuous_var_matrix(
    c, v, name="C"
)  # quantity of goods when you leave node i for vehicle k

# nodes constraints
for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[0, j, t, k] for j in range(1, c)) for t in range(len(d) - 1))
        <= 1
    )  # every vehicle is enabled to leave the first node (depot) at most once

for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[j, 0, t, k] for j in range(1, c)) for t in range(len(d) - 1))
        == mdl.sum(
            mdl.sum(x[0, j, t, k] for j in range(1, c)) for t in range(len(d) - 1)
        )
    )  # every vehicle that has left the depot must return to it

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
            for k in range(v)
        )
        == 1
    )  # every node must be visited exactly once

for k in range(v):
    for j in range(1, c):
        mdl.add_constraint(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
            == mdl.sum(
                mdl.sum(x[j, i, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
        )  # every node must be left from the vehicle that has visited it

# time
for k in range(v):
    for t in range(len(d) - 1):
        mdl.add_constraint(
            T[0, k] >= d[t] * 60 * mdl.sum(x[0, j, t, k] for j in range(1, c))
        )  # when a time interval to leave the depot is chosen, then each vehicle must leave the depot at that time or later

for k in range(v):
    for t in range(len(d) - 1):
        for j in range(1, c):
            mdl.add_constraint(
                T[j, k] >= d[t] * 60 * mdl.sum(x[i, j, t, k] for i in range(1, c))
            )  # same for the previous constraint but for the arrival time at node j

for k in range(v):
    for t in range(len(d) - 1):
        for j in range(1, c):
            mdl.add_constraint(
                T[j, k]
                <= d[t + 1] * 60 + (1 - mdl.sum(x[i, j, t, k] for i in range(1, c))) * M
            )  # when arriving in j at t, you ought to reach it before the end of the time interval

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                T[i, k] >= d[t] * 60 * mdl.sum(x[i, j, t, k] for j in range(0, c))
            )  # when you arrive at node j from node i at time t, you ought to depart from i in that time interval

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                T[i, k]
                <= d[t + 1] * 60 + (1 - mdl.sum(x[i, j, t, k] for j in range(0, c))) * M
            )  # when you arrive in j from i at time t, you ought to depart from i after the start of that time interval

for k in range(v):
    for i in range(c):
        for j in range(1, c):
            for t in range(len(d) - 1):
                if i != j:
                    mdl.add_constraint(
                        (1 - x[i, j, t, k]) * M + T[j, k] - T[i, k]
                        >= data[t][i][j] + t_mv * abs(q[j])
                    )
                    mdl.add_constraint(
                        (1 - x[i, j, t, k]) * M - T[j, k] + T[i, k]
                        >= -data[t][i][j] - t_mv * abs(q[j])
                    )  # if you go from i to j, then the difference between the arrival time at j and i should be the same as the time required to go from i to j
                    # plus the time to load/unload the required goods

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                (1 - x[i, 0, t, k]) * M + T[c, k] - T[i, k] >= data[t][i][0]
            )
            mdl.add_constraint(
                (1 - x[i, 0, t, k]) * M - T[c, k] + T[i, k] >= -data[t][i][0]
            )  # if you go from i to the depot, then the difference between the arrival time at the depot and i should be the same as the time required to go
            # from i to the depot

for k in range(v):
    for i in range(1, c):
        mdl.add_constraint(
            tw[i - 1][0] * 60 <= T[i, k]
        )  # you must arrive in i on-time (based on client side) lowest limit
        mdl.add_constraint(
            T[i, k] <= tw[i - 1][1] * 60
        )  # you must arrive in i on-time (based on client side) upper limit

for k in range(v):
    mdl.add_constraint(
        T[c, k] - T[0, k] <= 8 * 60
    )  # you must return to the depot within 8 hours (a shift)

# quantity constraints
for k in range(v):
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(1, c):
                if i != j:
                    mdl.add_constraint(
                        C[i, k] - C[j, k] + q[j] <= (1 - x[i, j, t, k]) * M
                    )
                    mdl.add_constraint(
                        C[j, k] - C[i, k] - q[j] <= (1 - x[i, j, t, k]) * M
                    )  # if you go from i to j, then the difference between the quantity of goods at i and j should be the same as the quantity
                    # of goods to deliver at j

for k in range(v):
    for i in range(c):
        mdl.add_constraint(
            max(0, q[i]) <= C[i, k]
        )  # if node i requires to load goods, you must leave i with at least the quantity of goods to collect
        mdl.add_constraint(
            C[i, k] <= min(Q, Q + q[i])
        )  # you must leave i with at most Q+q[i] goods if you have to deliver at node i goods or Q if you have to collect goods at node i

for k in range(v):
    del_qi = 0
    for j in range(1, c):
        if q[j] < 0:
            del_qi += q[j] * mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c)) for t in range(len(d) - 1)
            )
    mdl.add_constraint(
        C[0, k] >= del_qi
    )  # each vehicle must depart from the depot with at least the quantity of goods to deliver for its route

# objective function
mdl.minimize(
    mdl.sum(
        mdl.sum(
            mdl.sum(
                mdl.sum(x[i, j, t, k] * data[t][i][j] for i in range(c))
                for j in range(c)
            )
            for t in range(len(d) - 1)
        )
        for k in range(v)
    )
)  # minimize time of routes

sol = mdl.solve()

if sol is None:
    print("No solution found")

print("\n\n\nTotal time required is:")

print(sol.objective_value)

print(sol.objective_value + tot_t_mv)

print("\n\n\n")


counter_vehicles = 0
for k in range(v):
    counter_routes = 0
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(c):
                counter_routes += sol.get_value(x[i, j, t, k])
    if counter_routes == 0:
        print(f"vehicle {k} is NOT used")
    else:
        print(f"vehicle {k} is used")
        counter_vehicles += 1

print(f"Total number of used vehicles: {counter_vehicles}")

# print the route of every vechicle, with the time of arrival at every node, time wundow and quantity of goods
for k in range(v):
    print(f"Vehicle {k}")
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(c):
                if sol.get_value(x[i, j, t, k]) == 1:
                    if j != 0:
                        print(
                            f"From node {i} to node {j} at time {d[t]}:00 departing from {i} at {sol.get_value(T[i, k])} and arriving time at node {j} at {sol.get_value(T[j, k])}, time window {tw[j-1]}, quantity of goods {sol.get_value(C[i, k])}"
                        )
                    else:
                        print(
                            f"From node {i} to the depot at time {d[t]}:00 departing from {i} at {sol.get_value(T[i, k])} and returning at the depot at {sol.get_value(T[c,k])}, time window {tw[j-1]}, quantity of goods {sol.get_value(C[i, k])}"
                        )

Loading time matrix from file...
Loading time matrix from file...
Loading time matrix from file...
Loading time matrix from file...
Total time required to collect/deliver all goods: 95.0 minutes
Computing for 20 vehicles



Total time required is:
232.83999999999997
327.84




vehicle 0 is used
vehicle 1 is NOT used
vehicle 2 is NOT used
vehicle 3 is NOT used
vehicle 4 is NOT used
vehicle 5 is NOT used
vehicle 6 is NOT used
vehicle 7 is used
vehicle 8 is NOT used
vehicle 9 is NOT used
vehicle 10 is NOT used
vehicle 11 is used
vehicle 12 is NOT used
vehicle 13 is NOT used
vehicle 14 is NOT used
vehicle 15 is NOT used
vehicle 16 is NOT used
vehicle 17 is NOT used
vehicle 18 is NOT used
vehicle 19 is NOT used
Total number of used vehicles: 3
Vehicle 0
From node 0 to node 15 at time 10:00 departing from 0 at 652.500000001 and arriving time at node 15 at 666.9500000009971, time window [8, 12], quantity of goods 0
From node 7 to node 13 at time 11:00 departing from 7 at 682.3800000009901 and

In [53]:
from docplex.mp.model import Model
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import distance_matrix as dm
import pandas as pd
import os

TARGET_WEEKDAY = 0  # 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
TARGET_HOUR = 8  # 0 = 00:00, 1 = 01:00, ..., 23 = 23:00

load_dotenv()

api_key = os.getenv("GOOGLE_MAPS_API_KEY")

matrix_times, matrix_distance = dm.main(TARGET_WEEKDAY, TARGET_HOUR, api_key)

# parameters for time model
c = 20  # number of nodes
q = [
    0,
    10,
    10,
    -10,
    10,
    10,
    -10,
    10,
    10,
    -10,
    10,
    10,
    10,
    -10,
    10,
    10,
    10,
    10,
    -10,
    10,
]  # quantity of goods to collect/deliver  # quantity of goods to collect/deliver
Q = 300  # maximum capacity of the vehicle
distances = matrix_distance  # should match value of c
A = [(i, j) for i in range(c) for j in range(c) if i != j]
N0 = list(range(1, c))  # Nodi clienti (escluso il deposito)
L = 150  # Capacità massima della batteria [kWh]
a = {(i, j): 0.2 for (i, j) in A}  # Consumo base per arco
b = 1  # Consume dependent on the load [kWh/kg/km]
c_E = 0.2  # Cost per unit of consumed energy [€/kWh]
c_F = 15  # Fixed cost per vehicle (daily) [€/vehicle]
c_M = 0.2  # Cost per unit of physical distance [€/km]

del_q = 0  # quantity of goods to deliver

M = 100000  # big M

v = 20  # Number of vehicles

for i in range(len(distances)):
    for j in range(len(distances)):
        distances[i][j] = distances[i][j] / 1000  # Convert meters to kilometers

mdl_energy = Model("EnergyOptimization")

# Variabili di decisione

x_energy = mdl_energy.binary_var_cube(
    len(distances), len(distances), v, name="x"
)  # Percorsi
C = mdl_energy.continuous_var_matrix(
    range(c), range(v), name="C"
)  # quantity of goods when you leave node i for vehicle k
w = mdl_energy.continuous_var_list(v, name="w")  # Consumo massimo di energia

# Funzione obiettivo: Minimizzare i costi energetici

mdl_energy.minimize(
    mdl_energy.sum(
        [
            mdl_energy.sum([c_F * x_energy[0, j, k] for j in range(1, len(distances))])
            + mdl_energy.sum([c_M * distances[i][j] * x_energy[i, j, k] for i, j in A])
            + mdl_energy.sum(c_E * w[k] * x_energy[i, j, k] for i, j in A)
            for k in range(v)
        ]
    )
)

# Vincoli
for k in range(v):
    mdl_energy.add_constraint(
        mdl_energy.sum(x_energy[0, j, k] for j in N0) <= 1
    )  # each vehicle must leave the depot exactly once

for i in N0:
    mdl_energy.add_constraint(
        mdl_energy.sum(x_energy[i, j, k] for k in range(v) for j in range(c) if j != i)
        == 1
    )  # each node must be visited exactly once

for k in range(v):
    mdl_energy.add_constraint(
        mdl_energy.sum(x_energy[j, 0, k] for j in range(1, c))
        == mdl_energy.sum(x_energy[0, j, k] for j in range(1, c))
    )  # every vehicle that has left the depot must return to it

for i in range(len(distances)):
    for k in range(v):
        mdl_energy.add_constraint(
            mdl_energy.sum(x_energy[j, i, k] for j in range(len(distances)) if j != i)
            == mdl_energy.sum(
                x_energy[i, j, k] for j in range(len(distances)) if j != i
            )
        )

# quantity constraints
for k in range(v):
    for i in range(c):
        for j in range(1, c):
            if i != j:
                mdl_energy.add_constraint(
                    C[i, k] - C[j, k] + q[j] <= (1 - x_energy[i, j, k]) * M
                )
                mdl_energy.add_constraint(
                    C[j, k] - C[i, k] - q[j] <= (1 - x_energy[i, j, k]) * M
                )  # if you go from i to j, then the difference between the quantity of goods at i and j should be the same as the quantity
                # of goods to deliver at j

for k in range(v):
    for i in range(c):
        mdl_energy.add_constraint(
            max(0, q[i]) <= C[i, k]
        )  # if node i requires to load goods, you must leave i with at least the quantity of goods to collect
        mdl_energy.add_constraint(
            C[i, k] <= min(Q, Q + q[i])
        )  # you must leave i with at most Q+q[i] goods if you have to deliver at node i goods or Q if you have to collect goods at node i

for k in range(v):
    del_qi = 0
    for j in range(1, c):
        if q[j] < 0:
            del_qi += q[j] * mdl_energy.sum(x_energy[i, j, k] for i in range(c))
    print(del_qi)
    mdl_energy.add_constraint(
        C[0, k] >= -del_qi
    )  # each vehicle must depart from the depot with at least the quantity of goods to deliver for its route

for k in range(v):
    for row, col in A:
        mdl_energy.add_constraint(
            mdl_energy.sum(a[i, j] * x_energy[i, j, k] for i, j in A)
            + mdl_energy.sum(b * C[i, k] for i in range(c))
            <= w[k]
        )

for k in range(v):
    mdl_energy.add_constraint(0 <= w[k])
    mdl_energy.add_constraint(w[k] <= L)

# Risoluzione del modello

solution = mdl_energy.solve(log_output=False)

cost = solution.objective_value

if solution is None:
    print("Il modello non ha soluzione")
else:
    print("Costo totale:", solution.objective_value)
    for k in range(v):
        print(
            f"Consumo massimo del veicolo {k}: {solution.get_value(mdl_energy.get_var_by_name(f'w_{k}'))}"
        )

w = solution.get_values(w)

for w in w:
    if w == 0:
        unused_vehicle = True
        print("\nEsistono uno o più veicoli che non vengono utilizzati")
        break

for k in range(v):
    for i in range(c):
        for j in range(c):
            if solution.get_value(x_energy[i, j, k]) == 1:
                print(
                    f"Veicolo {k} va da {i} a {j} con un carico di {solution.get_value(C[i, k])} kg"
                )

Loading time matrix from file...
Total goods to deliver: 20
-10x_0_1_0-10x_0_3_0-10x_1_1_0-10x_1_3_0-10x_2_1_0-10x_2_3_0-10x_3_1_0-10x_3_3_0
-10x_0_1_1-10x_0_3_1-10x_1_1_1-10x_1_3_1-10x_2_1_1-10x_2_3_1-10x_3_1_1-10x_3_3_1
Costo totale: 48.972
Consumo massimo del veicolo 0: 16.0
Consumo massimo del veicolo 1: 14.0
Veicolo 0 va da 0 a 3 con un carico di 10.0 kg
Veicolo 0 va da 2 a 0 con un carico di 10.0 kg
Veicolo 0 va da 3 a 2 con un carico di 0 kg
Veicolo 1 va da 0 a 1 con un carico di 10.0 kg
Veicolo 1 va da 1 a 0 con un carico di 0 kg


In [ ]:
from docplex.mp.model import Model
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import distance_matrix as dm
import numpy as np
import pandas as pd
import os

TARGET_WEEKDAY = 0  # 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
TARGET_HOUR = 8  # 0 = 00:00, 1 = 01:00, ..., 23 = 23:00

load_dotenv()

api_key = os.getenv("GOOGLE_MAPS_API_KEY")

matrix_time_first_tw, matrix_distance = dm.main(TARGET_WEEKDAY, TARGET_HOUR, api_key)
matrix_time_second_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 1, api_key
)
matrix_time_third_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 2, api_key
)
matrix_time_fourth_tw, matrix_distance = dm.main(
    TARGET_WEEKDAY, TARGET_HOUR + 3, api_key
)


# sava datas in a matrix of matrixes
first_data = matrix_time_first_tw
second_data = matrix_time_second_tw
third_data = matrix_time_third_tw
fourth_data = matrix_time_fourth_tw
data = [first_data, second_data, third_data, fourth_data]

# parameters for time model
c = 20  # number of nodes
d = [8, 9, 10, 11, 12]  # time intervals
tw = [
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
    [8, 12],
]  # time windows
q = [
    0,
    10,
    10,
    -10,
    10,
    10,
    -10,
    10,
    10,
    -10,
    10,
    10,
    10,
    -10,
    10,
    10,
    10,
    10,
    -10,
    10,
]  # quantity of goods to collect/deliver
Q = 300  # maximum capacity of the vehicle
t_mv = 0.5  # time to move one item in/out from a vehicle [min]
M = 100000  # big M
tot_q = 0  # total quantity of goods to collect/delivery
del_q = 0  # quantity of goods to deliver
v = 3
# parameters for energy model
distances = matrix_distance  # should match value of c
print(distances)
A = [(i, j) for i in range(c) for j in range(c) if i != j]
N0 = list(range(1, c))  # Nodi clienti (escluso il deposito)
L = 150  # Capacità massima della batteria [kWh]
a = {(i, j): 0.2 for (i, j) in A}  # Consumo base per arco
b = 0.00005  # Consumo dipendente dal carico [kWh/kg/km]
c_E = 0.2  # Costo per unità di energia consumata [€/kWh]
c_F = 15  # Costo fisso per veicolo [€/vehicle]
c_M = 0.2  # Costo per unità di distanza percorsa [€/km]

unused_vehicle = False

for good in q:
    tot_q += abs(good)

tot_t_mv = tot_q * t_mv

print("Total time required to collect/deliver all goods:", tot_t_mv, "minutes")

for good in q:
    if good < 0:
        del_q += good
del_q = abs(del_q)

print(f"Computing for {v} vehicles")

# define model
mdl = Model("Pareto")

# decision variables of time model

x = mdl.binary_var_dict(
    (
        (i, j, t, k)
        for i in range(c)
        for j in range(c)
        for t in range(len(d) - 1)
        for k in range(v)
    ),
    name="x",
)  # x == 1 if it is true that we are going from node i to node j at time t with vehicle k
T = mdl.continuous_var_matrix(
    c + 1, v, name="t"
)  # arrival time at node i for vehicle k
C = mdl.continuous_var_matrix(
    c, v, name="C"
)  # quantity of goods when you leave node i for vehicle k

# decison variables of energy model

x_energy = [[[ 0 for _ in range(v)] for _ in range(c)] for _ in range(c)]

for k in range(v):
    for i in range(c):
        for j in range(c):
            for t in range(len(d)-1):
                x_energy[i][j][k] += x[i, j, t, k] 


w = mdl.continuous_var_list(v, name="w")  # max consuption of energy

#contraints of time model
# nodes constraints
for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[0, j, t, k] for j in range(1, c)) for t in range(len(d) - 1))
        == 1  
    )  # every vehicle must leave the depot exactly once, hence every vehicle is expoloited

for k in range(v):
    mdl.add_constraint(
        mdl.sum(mdl.sum(x[j, 0, t, k] for j in range(1, c)) for t in range(len(d) - 1))
        == 1
    )  # every vehicle that has left the depot must return to it

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
            for k in range(v)
        )
        == 1
    )  # every node must be visited exactly once

for k in range(v):
    for j in range(1, c):
        mdl.add_constraint(
            mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
            == mdl.sum(
                mdl.sum(x[j, i, t, k] for i in range(c) if i != j)
                for t in range(len(d) - 1)
            )
        )  # when you visit a node you ought to leave it

# time
for k in range(v):
    for t in range(len(d) - 1):
        mdl.add_constraint(
            T[0, k] >= d[t] * 60 * mdl.sum(x[0, j, t, k] for j in range(1, c))
        )  # when you leave depot at a time t, you ought to leave it at that time

for k in range(v):
    for t in range(len(d) - 1):
        for j in range(1, c):
            mdl.add_constraint(
                T[j, k] >= d[t] * 60 * mdl.sum(x[i, j, t, k] for i in range(1, c))
            )  # when you arrive at a node j from node i at time t, you ought to arrive at that time

for k in range(v):
    for t in range(len(d) - 1):
        for j in range(1, c):
            mdl.add_constraint(
                T[j, k]
                <= d[t + 1] * 60 + (1 - mdl.sum(x[i, j, t, k] for i in range(1, c))) * M
            )  # when arrive in j from i at time t, you ought to reach it before the end of the time interval

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                T[i, k] >= d[t] * 60 * mdl.sum(x[i, j, t, k] for j in range(0, c))
            )  # when you arrive at node j from node i at time t, you ought to depart in that time interval

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                T[i, k]
                <= d[t + 1] * 60 + (1 - mdl.sum(x[i, j, t, k] for j in range(0, c))) * M
            )  # when you arrive in j from i at time t, you ought to depart from i after the start of that time interval

for k in range(v):
    for i in range(c):
        for j in range(1, c):
            for t in range(len(d) - 1):
                if i != j:
                    mdl.add_constraint(
                        (1 - x[i, j, t, k]) * M + T[j, k] - T[i, k]
                        >= data[t][i][j] + t_mv * abs(q[j])
                    )
                    mdl.add_constraint(
                        (1 - x[i, j, t, k]) * M - T[j, k] + T[i, k]
                        >= -data[t][i][j] - t_mv * abs(q[j])
                    )  # if you go from i to j, then the difference between the arrival time at j and i should be the same as the time required to go from i to j
                    # plus the time to load/unload the required goods

for k in range(v):
    for t in range(len(d) - 1):
        for i in range(1, c):
            mdl.add_constraint(
                (1 - x[i, 0, t, k]) * M + T[c, k] - T[i, k] >= data[t][i][0]
            )
            mdl.add_constraint(
                (1 - x[i, 0, t, k]) * M - T[c, k] + T[i, k] >= -data[t][i][0]
            )  # if you go from i to the depot, then the difference between the arrival time at the depot and i should be the same as the time required to go
            # from i to the depot

for k in range(v):
    for i in range(1, c):
        mdl.add_constraint(
            tw[i - 1][0] * 60 <= T[i, k]
        )  # you must arrive in i on-time (based on client side) lowest limit
        mdl.add_constraint(
            T[i, k] <= tw[i - 1][1] * 60
        )  # you must arrive in i on-time (based on client side) upper limit

for k in range(v):
    mdl.add_constraint(
        T[c, k] - T[0, k] <= 8 * 60
    )  # you must return to the depot within 8 hours (a shift)

# quantity constraints
for k in range(v):
    for t in range(len(d) - 1):
        for i in range(c):
            for j in range(1, c):
                if i != j:
                    mdl.add_constraint(
                        C[i, k] - C[j, k] + q[j] <= (1 - x[i, j, t, k]) * M
                    )
                    mdl.add_constraint(
                        C[j, k] - C[i, k] - q[j] <= (1 - x[i, j, t, k]) * M
                    )  # if you go from i to j, then the difference between the quantity of goods at i and j should be the same as the quantity
                    # of goods to deliver at j

for k in range(v):
    for i in range(c):
        mdl.add_constraint(
            max(0, q[i]) <= C[i, k]
        )  # if node i requires to load goods, you must leave i with at least the quantity of goods to collect
        mdl.add_constraint(
            C[i, k] <= min(Q, Q + q[i])
        )  # you must leave i with at most Q+q[i] goods if you have to deliver at node i goods or Q if you have to collect goods at node i

for k in range(v):
    del_qi = 0
    for j in range(1, c):
        if q[j] < 0:
            del_qi += q[j] * mdl.sum(
                mdl.sum(x[i, j, t, k] for i in range(c)) for t in range(len(d) - 1)
            )
    mdl.add_constraint(
        C[0, k] >= -del_qi
    )  # each vehicle must depart from the depot with at least the quantity of goods to deliver for its route

# Convert meters to kilometers

for i in range(len(distances)):
    for j in range(len(distances)):
        distances[i][j] = distances[i][j] / 1000 


#constraints of energy model

for i in N0:
    mdl.add_constraint(
        mdl.sum(
            x_energy[i][j][k] for k in range(v) for j in range(len(distances)) if j != i
        )
        == 1
    )   #forse doppione

for i in range(len(distances)):
    mdl.add_constraint(
        mdl.sum(x_energy[j][i][k] for j in range(len(distances)) if j != i)
        == mdl.sum(x_energy[i][j][k] for j in range(len(distances)) if j != i)
    )  #forse doppione

for k in range(v):
    for row,col in A:
        mdl.add_constraint(
            mdl.sum(a[i, j] * x_energy[i][j][k]  for i, j in A) + mdl.sum(b * C[i,k] for i in range(c))
            == w[k] 
        )

for k in range(v):
    mdl.add_constraint(0 <= w[k])
    mdl.add_constraint(w[k] <= L)


def calc_obj_cost(x_energy, w, distances, A, c_F, c_M, c_E, v):
    """
    Computes the summation for the energy-cost based objective function.
    """
    total_sum = 0
    for k in range(v):
        sum_energy = sum(c_F * x_energy[0][j][k] for j in range(1, len(distances)))
        sum_distances = sum(c_M * distances[i][j]*x_energy[i][j][k] for i, j in A)
        sum_weight = c_E * w[k]
        total_sum += sum_energy + sum_distances + sum_weight
    return total_sum

def calc_obj_time(x, data, c, d, v):
    """
    Computes the summation for the time based objective function.
    """
    total_sum = 0
    for k in range(v):
        for t in range(len(d) - 1):
            for j in range(c):
                for i in range(c):
                    total_sum += x[i][j][t][k] * data[t][i][j]
    return total_sum


alpha_values = np.linspace(0, 1, 10)

# list to memorize costs and times
times = []
costs = []
times_a=[[]]
costs_a = [[]]

mdl.parameters.timelimit = 60*50

#resolv the model for different alpha values to get the paretro front
for a in alpha_values:
    mdl.minimize(
        a * 
        mdl.sum(
            [
                mdl.sum([c_F * x_energy[0][j][k] for j in range(1, len(distances))])
                + mdl.sum([c_M * distances[i][j]*x_energy[i][j][k]  for i, j in A])
                + c_E * w[k]
                for k in range(v)
            ]
        ) + 
        (1 - a) * 
        mdl.sum(
            mdl.sum(
                mdl.sum(
                    mdl.sum(x[i, j, t, k] * data[t][i][j] for i in range(c))
                    for j in range(c)
                )
                for t in range(len(d) - 1)
            )
            for k in range(v)
        )
    )
    
    solution = mdl.solve(log_output=False)
    if solution is None:
        print("Il modello non ha soluzione con alpha",a)
    else:
        # get values of decision variables  
        # Create a zero-initialized multidimensional list to save the result of x_energy

        x_energy_result = [[[ 0 for _ in range(v)] for _ in range(c)] for _ in range(c)]

        for k in range(v):
            for i in range(c):
                for j in range(c):
                    for t in range(len(d)-1):
                        x_energy_result[i][j][k] += solution.get_value(x[i, j, t, k])
        
        # Create a zero-initialized multidimensional list to save the result of x
        x_t =[
            [
                [
                    [0 for k in range(v)]
                    for t in range(len(d) - 1)
                ]
                for j in range(c)
            ]
            for i in range(c)
        ]

        for k in range(v):
            for t in range(len(d)-1):
                for i in range(c):
                    for j in range(c):
                        x_t[i][j][t][k] = solution.get_value(x[i, j, t, k])


        w_solution = solution.get_values(w)
        print("a: ",a)
        print("w: ",w_solution)

        # Calculate time and cost
        time = calc_obj_time(x_t, data, c, d, v)
        cost = calc_obj_cost(x_energy_result, w_solution, distances, A, c_F, c_M, c_E, v)
        
        # Memorize the result
        times.append(time)
        costs.append(cost)
        times_a.append([time,a])
        costs_a.append([cost,a])
        print()

        # print the route of every vechicle, with the time of arrival at every node, time wundow and quantity of goods
        for k in range(v):
            print(f"Vehicle {k}")
            for t in range(len(d) - 1):
                for i in range(c):
                    for j in range(c):
                        if solution.get_value(x[i, j, t, k]) == 1:
                            if j != 0:
                                print(
                                    f"From node {i} to node {j} at time {d[t]}:00 departing from {i} at {solution.get_value(T[i, k])} and arriving time at node {j} at {solution.get_value(T[j, k])}, time window {tw[j-1]}, quantity of goods {solution.get_value(C[i, k])}"
                                )
                            else:
                                print(
                                    f"From node {i} to the depot at time {d[t]}:00 departing from {i} at {solution.get_value(T[i, k])} and returning at the depot at {solution.get_value(T[c,k])}, time window {tw[j-1]}, quantity of goods {solution.get_value(C[i, k])}"
                                )

        # for k in range(v):
        #     for i in range(c):
        #         #if solution.get_value(C[i,k])!=0:
        #             print(
        #                 f"C[{i},{k}]:{solution.get_value(C[i,k])}"
        #                 )


# print the result
plt.figure(figsize=(10, 6))
#plt.plot(times, costs, color='blue', linewidth=2)  #line plot
plt.scatter(times, costs, color='blue', s=5, alpha=0.8)  # point plot
plt.title("Pareto Front: Times vs Cost")
plt.xlabel("Times")
plt.ylabel("Cost")
plt.grid(True)
plt.show()
print("times",times)
print(times_a)
print(len(times))
print("costs",costs)
print(len(costs))
print(costs_a)

Loading time matrix from file...
Loading time matrix from file...
Loading time matrix from file...
Loading time matrix from file...
[[    0.  3448.  3279.  3779.  4910.  4623.  2733.  4373.  2921.  5003.
   4158.  4970.  4629.  1184.  6568.  2130.  3088.  7292.  2499.  3012.]
 [ 2219.     0.  2048.  2548.  3679.  3392.  3253.  5470.  2689.  4170.
   2927.  3645.  3399.  2279.  4438.  3500.  2942.  6354.  2397.  2866.]
 [ 5046.  3542.     0.  2153.  1631.  1344.  5768. 11684.  4358.  2122.
    879.  2125.  1351.  4339.  2390.  5857.  4525.  4306.  3722.  4449.]
 [ 4525.  2516.  1319.     0.   910.   968.  5023. 11374.  4464.  2321.
   2250.  2956.   630.  4571.  2371.  6089.  4406.  4287.  2695.  4517.]
 [ 4157.  5769.  6708.  6211.     0.  5422.  6427.  7691.  5958.  3604.
   5470.  6430.  5429.  3469.  5189.  4967.  6231.  5914.  3733.  6155.]
 [ 4468.  2458.  1262.   809.  1073.     0.  4966.  7718.  4181.  2483.
   2141.  2899.   793.  4733.  2533.  5748.  4348.  4449.  2638.  4272.